# CS246 - Homework 1
## Question 1


### Setup

Let's setup Spark on your Colab environment.  Run the cell below!

In [1]:
!pip install pyspark
!pip install -U -q PyDrive2
#the output 'xxx is not a symbolic link' will not affect your implementation or execution
#to fix 'xxx is not a symbolic link', you can uncomment the lines starting from !mv xxxx
#you may need to replace xxx.11 with the correct version if other errors come up after colab update
#to get the correct version, use !ls /usr/local/lib to find out
!mv /usr/local/lib/libtbbmalloc_proxy.so.2 /usr/local/lib/libtbbmalloc_proxy.so.2.backup
!mv /usr/local/lib/libtbbmalloc.so.2 /usr/local/lib/libtbbmalloc.so.2.backup
!mv /usr/local/lib/libtbbbind_2_5.so.3 /usr/local/lib/libtbbbind_2_5.so.3.backup
!mv /usr/local/lib/libtbb.so.12 /usr/local/lib/libtbb.so.12.backup
!mv /usr/local/lib/libtbbbind_2_0.so.3 /usr/local/lib/libtbbbind_2_0.so.3.backup
!mv /usr/local/lib/libtbbbind.so.3 /usr/local/lib/libtbbbind.so.3.backup
!ln -s /usr/local/lib/libtbbmalloc_proxy.so.2.11 /usr/local/lib/libtbbmalloc_proxy.so.2
!ln -s /usr/local/lib/libtbbmalloc.so.2.11 /usr/local/lib/libtbbmalloc.so.2
!ln -s /usr/local/lib/libtbbbind_2_5.so.3.11 /usr/local/lib/libtbbbind_2_5.so.3
!ln -s /usr/local/lib/libtbb.so.12.11 /usr/local/lib/libtbb.so.12
!ln -s /usr/local/lib/libtbbbind_2_0.so.3.11 /usr/local/lib/libtbbbind_2_0.so.3
!ln -s /usr/local/lib/libtbbbind.so.3.11 /usr/local/lib/libtbbbind.so.3
#If error related to the above execution occurs, you can try commenting out the above 12 lines under pip install PyDrive2 (not included)

# !sudo ldconfig
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=680c29eceb773bc19795dd91d8086e5b16e528f766fd47e31568f320ef270415
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.3 MB/s eta 0:00:00
The following additional packages will be installed:
  libxtst6 openjdk-8-jre-headless
Suggested packages:
  openjdk-8-demo openjdk-8-source libnss-mdns fonts-dejavu-extra fonts-nanum fonts-ipafont-gothic
  fonts-ipafont-mincho fonts-wqy-microhei fonts-wqy-zenhei fonts-indic
The following NEW packages will be installed:
  libxtst6 openjdk-8-jdk-headless openjdk-8-jre-headless
0 upgraded, 3 newly installed, 0 to remove and 30 not upgraded.
Need to get 39.7 MB of archives.
After this operation, 144 M

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.

**Make sure to follow the interactive instructions.**

In [2]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1Qaa1YlL_ihEBv_36OECIvK8_pSKWT7Sj'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('soc-LiveJournal1Adj.txt')

In [4]:
# Let's import the libraries we will need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

Let's initialize the Spark context.


In [5]:
# create the Spark Session
spark = SparkSession.builder.getOrCreate()

# create the Spark Context
sc = spark.sparkContext

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

# Question 1

First read the document.

In [7]:
txt = spark.read.text("soc-LiveJournal1Adj.txt")

# Split the txt file data into id and friends columns
data = txt.select(split(col("value"), "\t").alias("split_row"))\
          .select(col("split_row")[0].alias("id"), split(col("split_row")[1], ",").alias("friends"))\
          .select("id", explode("friends").alias("friend"))

# Count number of occurances of friends of friends
count_data = data.alias("d1").join(data.alias("d2"), "friend")\
                 .filter(col("d1.id") != col("d2.id"))\
                 .groupBy("d1.id", "d2.id")\
                 .agg(count("*").alias("count"))

# Rank number of occurances and keep top 10 for each id
spec = Window.partitionBy("d1.id").orderBy(desc("count"), "d2.id")
top10 = count_data.withColumn("rank", rank().over(spec))\
                  .filter(col("rank") <= 10)\
                  .groupBy("d1.id")\
                  .agg(collect_list("d2.id").alias("recommendations"))\
                  .orderBy("d1.id")

In [8]:
ids = [11, 924, 8941, 8942, 9019, 9020, 9021, 9022, 9990, 9992, 9993]
filtered_results = top10.filter(col("d1.id").isin(ids))
filtered_results.show(truncate=False)


+----+----------------------------------------------------------------------+
|id  |recommendations                                                       |
+----+----------------------------------------------------------------------+
|11  |[27552, 27573, 27574, 27589, 27590, 27600, 27617, 27620, 27667, 32072]|
|8941|[8938, 8942, 8946, 8939, 8943, 8944, 8945, 8940]                      |
|8942|[8938, 8939, 8941, 8945, 8946, 8940, 8943, 8944]                      |
|9019|[320, 9018, 9016, 9017, 9020, 9021, 9022, 317, 9023]                  |
|9020|[9021, 320, 9016, 9017, 9018, 9019, 9022, 317, 9023]                  |
|9021|[9020, 320, 9016, 9017, 9018, 9019, 9022, 317, 9023]                  |
|9022|[9019, 9020, 9021, 317, 320, 9016, 9017, 9018, 9023]                  |
|924 |[11860, 15416, 2409, 43748, 439, 45881, 6995]                         |
|9990|[9987, 9988, 9989, 9993, 9994, 35667, 9991, 9992, 13134, 13478]       |
|9992|[9987, 9989, 35667, 9988, 9990, 9993, 9994, 9991]         